In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import EdgePooling, GraphConv, BatchNorm

import import_ipynb
from vision_transformer import ViT

In [2]:
class FusionModel(nn.Module):
    def __init__(self, nfeat, nhid, nclass, depth, image_size, patch_size, heads, dropout):
        super().__init__()
        
        self.gcn = GraphConv(nhid, nhid)
        self.vit = ViT(image_size=image_size*5,
                       patch_size=patch_size, num_classes=nclass,
                       dim=nhid, depth=depth,
                       heads=heads, mlp_dim=nhid,
                       dropout=dropout, emb_dropout=dropout,
                       channels=nfeat
                      )
        
#         self.batch_conv1 = nn.BatchNorm2d(5)
#         self.batch_conv2 = nn.BatchNorm2d(5)
        
#         self.batch1 = BatchNorm(nhid)
#         self.batch2 = BatchNorm(nhid)
#         self.batch3 = BatchNorm(nhid)
        
        self.linear_in = nn.Sequential(nn.Linear(103, nhid))
        self.linear_out = nn.Sequential(nn.Linear(2*nhid, nclass))
        
    def forward(self, x, adj, features, slices):
        
        print(f"Shape of x before linear in: {x.shape}")
        x = self.linear_in(x.float())
        print(f"Shape of x after linear in: {x.shape}")
        
        print(f"adj shape: {adj.shape}")
        print(f"features shape: {features.shape}")
        
        x = x.relu()
        x = F.dropout(x, p=0.2, training=self.training)
        
        x_gcn = self.gcn(x, adj.long(), features)
        print(f"Shape of x_gcn in : {x_gcn.shape}")
        
        x_vit = self.vit(slices.transpose(1, 3).float())
        print(f"Shape of x_vit in: {x_vit.shape}")
        
        x = torch.cat([x_gcn, x_vit], dim=1)
        print(f"Shape after concatention: {x.shape}")
        
        x = self.linear_out(x)
        print(f"Final output shape: {x.shape}")
        
        return x